In [ ]:
# install once
!pip install -U boto3 sagemaker awscli
# restart jupyter kernel

In [2]:
import sagemaker
import boto3, os
from sagemaker import get_execution_role

sess = sagemaker.Session()
role = get_execution_role()
sagemaker_default_bucket = sess.default_bucket()
region = sess.boto_session.region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.estimator import Estimator

# https://github.com/aws/deep-learning-containers/blob/master/available_images.md
image_uri = f'763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:2.2.0-gpu-py310-cu121-ubuntu20.04-sagemaker'
# image_uri = f'763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:2.3.0-gpu-py311-cu121-ubuntu20.04-sagemaker'

instance_type = "ml.g5.12xlarge"    # 4 * A10g (24G/GPU)
# instance_type = "ml.g5.48xlarge"    # 8 * A10g (24G/GPU)
# instance_type = "ml.p4d.24xlarge"   # 8 * A100 (40G/GPU)
# instance_type = "ml.p5.48xlarge"    # 8 * H100 (80G/GPU)

instance_count = 2                  # Multi-node

envs = {
    # 'MODEL_ID_OR_S3_PATH': 's3://llm-artifacts-us-east-1/Qwen2-7B-Instruct/*',
    "DATA_S3_PATH": f's3://{sagemaker_default_bucket}/qwen2-train-dataset/*',
    'MODEL_ID_OR_S3_PATH': f's3://{sagemaker_default_bucket}/Qwen2-0.5B-Instruct/*',
    'MODEL_SAVE_PATH_S3': f's3://{sagemaker_default_bucket}/output-model/2406/'
}

hypers = {
}

smp_estimator = Estimator(role=role,
    sagemaker_session=sess,
    base_job_name='sm-qwen2-multinode',
    entry_point="estimator_entry.py",
    source_dir='submit_src/',
    instance_type=instance_type,
    instance_count=instance_count,
    environment=envs,
    hyperparameters=hypers,
    image_uri=image_uri,
    input_mode="FastFile",
    # output_path=s3_output_bucket,
    # checkpoint_s3_uri=checkpoint_s3_uri,
    max_run=7200,
    keep_alive_period_in_seconds=3600,
    enable_remote_debug=True,
    disable_output_compression=True,
)

smp_estimator.fit()

INFO:sagemaker:Creating training-job with name: sm-qwen2-multinode-2024-07-11-00-25-58-687


2024-07-11 00:26:02 Starting - Starting the training job...
2024-07-11 00:26:02 Pending - Training job waiting for capacity......
2024-07-11 00:27:20 Pending - Preparing the instances for training...
2024-07-11 00:27:59 Downloading - Downloading input data...
2024-07-11 00:28:09 Downloading - Downloading the training image.....................
2024-07-11 00:31:42 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-07-11 00:32:07,217 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-07-11 00:32:07,252 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-07-11 00:32:07,263 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-07-11 00:32:07,265 sagemaker_pytorch_container.training INFO     Invoking user training script.
bash: cannot

In [ ]:
!aws s3 ls s3://$sagemaker_default_bucket/output-model/2406/